In [ ]:
import argparse
import yaml
import torch
import torch.nn as nn
import wandb
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from torchvision.models import resnet18
from utils import DataManager, get_model, get_pretrain_transform, get_classifier, get_callbacks_logger, get_checkpoint, prepare_transforms
from continuum.metrics.logger import Logger as LOGGER
from lab.models.simclr import  SimCLRLab


class Depth3Width32Net(nn.Module):
    def __init__(self, input_dim=784, output_dim=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)  # Layer 1
        self.fc2 = nn.Linear(32, 32)         # Layer 2
        self.fc3 = nn.Linear(32, output_dim)  # Layer 3 (logits)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Flatten input
        out = self.fc1(x)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        return out

def main(args):
    seeds = args.seeds
    class_increment = int(args.num_classes / args.num_tasks)

    for scenario_id in tqdm(seeds, desc="Scenerio"):

        seed_everything(scenario_id)
        torch.set_float32_matmul_precision(args.set_float32_matmul_precision)

        backbone = Depth3Width32Net(input_dim=args.input_dim, output_dim=args.feature_dim)
        model = SimCLRLab(backbone, args)

        #pretrain_transform = get_pretrain_transform(args)
        transform, val_transform = prepare_transforms(dataset=args.dataset)
        def pretrain_transform(x):
            view1 = transform(x)
            view2 = transform(x)
            val = val_transform(x)
            return (view1, view2, val)
        logger = LOGGER()

        data_manager = DataManager(
            args=args,
        )

        train_pretrain_scenario, train_classifier_scenario, test_classifier_scenario = data_manager.get_scenerio(scenario_id)

        for task_id, train_pretrain_taskset in tqdm(enumerate(train_pretrain_scenario), desc="Training tasks"):
            print("TASK ID", task_id)
            train_classifier_taskset = train_classifier_scenario[:task_id+1]
            test_classifier_taskset = test_classifier_scenario[:task_id+1]
            train_pretrain_loader, train_classifier_loader, test_classifier_loader = data_manager.get_dataloader(
                train_pretrain_taskset, 
                train_classifier_taskset, 
                test_classifier_taskset, 
                pretrain_transform,
                args
            )

            _, pretrain_wandb_logger = get_callbacks_logger(args, training_type="pretrain", task_id=task_id, scenario_id=scenario_id)

            if task_id>0:
                model = SimCLRLab(model.backbone, args)

            '''
            Train the model
            '''
            trainer = pl.Trainer(
                max_epochs=args.train_epochs, 
                accelerator=args.accelerator,
                devices=args.gpu_devices,
                accumulate_grad_batches=args.train_accumulate_grad_batches,
                #callbacks=pretrain_callbacks,
                logger=pretrain_wandb_logger,
                enable_checkpointing=False,
                strategy=args.strategy,
                precision=args.precision,
                sync_batchnorm=args.sync_batchnorm,
            )
            trainer.fit(model, train_pretrain_loader)

            # load the best model
            #model = get_checkpoint(trainer, backbone, args)

            '''
            Evaluate the model
            '''
            _, classifier_wandb_logger = get_callbacks_logger(args, training_type="classifier", task_id=task_id, scenario_id=scenario_id)

            classifier = get_classifier(
                model.backbone, 
                num_classes=class_increment*(task_id+1),
                logger=logger,
                args=args
            )

            trainer = pl.Trainer(
                max_epochs=args.test_epochs, 
                accelerator=args.accelerator,
                devices=args.gpu_devices,
                enable_checkpointing=False,
                logger=classifier_wandb_logger,
                strategy=args.strategy,
                precision=args.precision,
                sync_batchnorm=args.sync_batchnorm,
            )
            trainer.fit(classifier, train_classifier_loader, test_classifier_loader)

            wandb.finish()




from types import SimpleNamespace

config_name = "simclr_mnist"
with open(f"lab/config/{config_name}.yaml", 'r') as file:
    config_dict = yaml.safe_load(file)

# Convert dict to an object
config = SimpleNamespace(**config_dict)
main(config)

/home/medici/Documents/Projects/CSSL/CSSL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/medici/Documents/Projects/CSSL/CSSL/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Scenerio:   0%|          | 0/1 [00:00<?, ?it/s]Seed set to 5
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


TASK ID 0


wandb: Currently logged in as: mxdici (mxdici-jozi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type                 | Params | Mode 
-----------------------------------------------------------------
0 | backbone        | Depth3Width32Net     | 27.2 K | train
1 | projection_head | SimCLRProjectionHead | 4.3 K  | train
2 | criterion       | NTXentLoss           | 0      | train
-----------------------------------------------------------------
31.5 K    Trainable params
0         Non-trainable params
31.5 K    Total params
0.126     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Epoch 20:   0%|          | 0/50 [00:00<?, ?it/s, v_num=ivvj, train_loss_step=2.820, representation_std_step=0.171, regions=9.360, train_loss_epoch=4.660, representation_std_epoch=0.169]         

In [1]:
import argparse
import yaml
import torch
import torch.nn as nn
import wandb
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from torchvision.models import resnet18
from utils import DataManager, get_model, get_pretrain_transform, get_classifier, get_callbacks_logger, get_checkpoint, prepare_transforms
from continuum.metrics.logger import Logger as LOGGER
from lab.models.byol import  BYOLLab


class Depth3Width32Net(nn.Module):
    def __init__(self, input_dim=784, output_dim=10):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)  # Layer 1
        self.fc2 = nn.Linear(32, 32)         # Layer 2
        self.fc3 = nn.Linear(32, output_dim)  # Layer 3 (logits)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Flatten input
        out = self.fc1(x)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        return out

def main(args):
    seeds = args.seeds
    class_increment = int(args.num_classes / args.num_tasks)

    for scenario_id in tqdm(seeds, desc="Scenerio"):

        seed_everything(scenario_id)
        torch.set_float32_matmul_precision(args.set_float32_matmul_precision)

        backbone = Depth3Width32Net(input_dim=args.input_dim, output_dim=args.feature_dim)
        model = BYOLLab(backbone, args)

        #pretrain_transform = get_pretrain_transform(args)
        transform, val_transform = prepare_transforms(dataset=args.dataset)
        def pretrain_transform(x):
            view1 = transform(x)
            view2 = transform(x)
            val = val_transform(x)
            return (view1, view2, val)
        logger = LOGGER()

        data_manager = DataManager(
            args=args,
        )

        train_pretrain_scenario, train_classifier_scenario, test_classifier_scenario = data_manager.get_scenerio(scenario_id)

        for task_id, train_pretrain_taskset in tqdm(enumerate(train_pretrain_scenario), desc="Training tasks"):
            print("TASK ID", task_id)
            train_classifier_taskset = train_classifier_scenario[:task_id+1]
            test_classifier_taskset = test_classifier_scenario[:task_id+1]
            train_pretrain_loader, train_classifier_loader, test_classifier_loader = data_manager.get_dataloader(
                train_pretrain_taskset, 
                train_classifier_taskset, 
                test_classifier_taskset, 
                pretrain_transform,
                args
            )

            _, pretrain_wandb_logger = get_callbacks_logger(args, training_type="pretrain", task_id=task_id, scenario_id=scenario_id)

            if task_id>0:
                model = BYOLLab(model.backbone, args)

            '''
            Train the model
            '''
            trainer = pl.Trainer(
                max_epochs=args.train_epochs, 
                accelerator=args.accelerator,
                devices=args.gpu_devices,
                accumulate_grad_batches=args.train_accumulate_grad_batches,
                #callbacks=pretrain_callbacks,
                logger=pretrain_wandb_logger,
                enable_checkpointing=False,
                strategy=args.strategy,
                precision=args.precision,
                sync_batchnorm=args.sync_batchnorm,
            )
            trainer.fit(model, train_pretrain_loader)

            # load the best model
            #model = get_checkpoint(trainer, backbone, args)

            '''
            Evaluate the model
            '''
            _, classifier_wandb_logger = get_callbacks_logger(args, training_type="classifier", task_id=task_id, scenario_id=scenario_id)

            classifier = get_classifier(
                model.backbone, 
                num_classes=class_increment*(task_id+1),
                logger=logger,
                args=args
            )

            trainer = pl.Trainer(
                max_epochs=args.test_epochs, 
                accelerator=args.accelerator,
                devices=args.gpu_devices,
                enable_checkpointing=False,
                logger=classifier_wandb_logger,
                strategy=args.strategy,
                precision=args.precision,
                sync_batchnorm=args.sync_batchnorm,
            )
            trainer.fit(classifier, train_classifier_loader, test_classifier_loader)

            wandb.finish()




from types import SimpleNamespace

config_name = "byol_mnist"
with open(f"lab/config/{config_name}.yaml", 'r') as file:
    config_dict = yaml.safe_load(file)

# Convert dict to an object
config = SimpleNamespace(**config_dict)
main(config)

/home/medici/Documents/Projects/CSSL/CSSL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/medici/Documents/Projects/CSSL/CSSL/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Scenerio:   0%|          | 0/1 [00:00<?, ?it/s]Seed set to 5
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


TASK ID 0


wandb: Currently logged in as: mxdici (mxdici-jozi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name                     | Type                     | Params | Mode 
------------------------------------------------------------------------------
0 | backbone                 | Depth3Width32Net         | 27.2 K | train
1 | projection_head          | BYOLProjectionHead       | 4.3 K  | train
2 | prediction_head          | BYOLPredictionHead       | 4.3 K  | train
3 | backbone_momentum        | Depth3Width32Net         | 27.2 K | train
4 | projection_head_momentum | BYOLProjectionHead       | 4.3 K  | train
5 | criterion                | NegativeCosineSimilarity | 0      | train
------------------------------------------------------------------------------
35.7 K    Trainable params
31.5 K    Non-trainable params
67.2 K    Total params
0.269     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 50/50 [00:00<00:00, 58.60it/s, v_num=p2xi, train_loss_step=-3.56, representation_std_step=0.126, train_loss_epoch=-3.45, representation_std_epoch=0.125, regions=5.400]

...

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 235/235 [00:02<00:00, 105.19it/s, v_num=ab8r, train_loss=5.190, val_loss=3.110, Task Accuracy=0.501, Accuracy=0.509, AIC=0.716, BWT=-0.0992, FWT=0.638, PBWT=0.000, Remembering=0.901, Forgetting=0.192]


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


AIC,▄▄▆▁▅▂▄▅▃▃▅▆▆▅▄▅▁▅▇▇█▇▇▇▇
Accuracy,▄▄▆▁▅▂▄▅▃▃▅▆▆▅▄▅▁▅▇▇█▇▇▇▇
BWT,▁▄▅▂▅▁▄▃▄▆▅▅▄▅▆▄▂▃███▇▇▇▇
FWT,▁▅▅▁▆▃▅▃▄▄▆▆▆▆▅▄▂▅▇▇█▇▇█▇
Forgetting,█▅▄▇▄█▅▆▅▃▄▄▅▄▃▅▇▆▁▁▁▂▂▂▂
PBWT,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Remembering,▁▄▅▂▅▁▄▃▄▆▅▅▄▅▆▄▂▃███▇▇▇▇
Task Accuracy,█▂▅▁▂▁▁█▁▁▁▄▅▃▂▄▂▅▄▅▆▅▆▅▅
epoch,▁▁▁▂▃▄▄▅▅▆▆▆▆▇▇█▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄
regions,█▄█▇▆█▄▁▄▇▇▆▇█▇██▇▇█▇▇▇▆▆▆▇▇▆▆▆▇▇▇▆▇▇▇▇▇
representation_std_epoch,▆▆██▇▅▄▄▃▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁


Training tasks: 5it [19:06, 229.27s/it]
Scenerio: 100%|██████████| 1/1 [19:06<00:00, 1146.39s/it]
